### 이미지 전처리
- 이미지의 색상 데이터를 분석해서 수치하는 작업
- 이미지들은 내부적으로 사용하는 색상 체계와 크기 등이 다를 수 있다.
- 이 때문에 머신러닝에서 학습을 할 때 데이터가 통일되어 있지 않기 때문에 학습이 어려워진다.
- 색상 체계, 크기 등을 일정하게 맞추는 작업을 해야한다.

In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid') # sns에 흰색 그리드 유지
import missingno # 결측치 시각화

# KFold (교차 검증을 사용하기 위해)
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증 데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가 함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier


# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import  XGBRegressor

# 머신러닝 알고리즘 - 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 머신러닝 알고리즘 - 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 다중분류를 위한 원핫 인코더
from keras.utils import to_categorical

# 학습 자동 중단
from keras.callbacks import EarlyStopping

# 모델 저장
from keras.callbacks import ModelCheckpoint

# 저장된 딥러닝 모델 불러오기
from keras.models import load_model

# 딥러닝
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

from keras.utils import np_utils
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Reshape
from keras.layers import UpSampling2D
from keras.layers import Input
from keras.models import Model

# 이미지 조작을 위한
from PIL import Image

import os, glob

# 이미지 생성자
from keras.preprocessing.image import ImageDataGenerator

# VGG16 모델 (이미 학습이 완료되어 있는 이미지 인식 모델)
from keras.applications import VGG16

# 자연어 처리
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

# 전처리, 활성 함수 셋팅
from keras import optimizers, initializers, regularizers, metrics

# 저장
import pickle

# 시간 모듈
import time

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'   # 윈도우용
# plt.rcParams['font.family'] = 'AppleGothic'   # 맥용
plt.rcParams['font.size'] = 10                 # 폰트 크기
plt.rcParams['figure.figsize'] = 15,8          # 그래프 크기
plt.rcParams['axes.unicode_minus'] = False     # - 기호 깨짐 방지


# 경고 메시지가 안나오게 하기
import warnings
warnings.filterwarnings('ignore')

# gpu 사용 초기화 및 할당
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [2]:
# 사용할 이미지가 들어있는 폴더 위치
root_dir = '../images/101_new'

In [3]:
# 폴더 이름들을 가져와 결과데이터로 구성한다.
a1 = os.walk(root_dir)
categories = list(a1)[0][1]

In [4]:
# 결과 데이터의 개수를 담는다.
nb_classes = len(categories)
nb_classes

102

In [5]:
# 입력과 결과데이터를 담을 리스트
# 입력 데이터 (이미지 데이터)
X=[]
# 결과 데이터 (폴더이름을 숫자로 변환한 것)
Y=[]

In [6]:
# 제공되는 이미지의 크기가 다양하므로 한가지 사이즈로 변환해야한다.
# 변환될 가로, 세로 사이즈 지정
image_w = 64
image_h = 64

In [7]:
# 폴더의 수만큼 반복한다.
for idx, cat in enumerate(categories):
    # 이미지를 읽어올 폴더의 이름을 구성한다.
    image_dir = root_dir + '/' + cat
    # print(image_dir)
    
    # 해당 폴더에 있는 파일 이름들을 가져온다.
    files = glob.glob(image_dir + '/*')
    # print(files)
    
    print(f'{cat} 처리중')
    
    # 파일의 수만큼 반복한다.
    for i, f in enumerate(files):
        # 파일의 데이터를 읽어온다.
        img = Image.open(f)
        # 색상 모드를 변경한다. ('L' :흑백,'RGB':컬러)
        img = img.convert('RGB')
        # 이미지의 크기를 변경한다.
        img = img.resize((image_w, image_h))
        # plt.imshow(img)
        # plt. show()
        
        # 이미지 데이터를 ndarray 형태로 생성한다.
        data = np.asarray(img)
        #print(data)
        
        # 데이터를 담는다.
        X.append(data)
        Y.append(idx)
    

accordion 처리중
airplanes 처리중
anchor 처리중
ant 처리중
BACKGROUND_Google 처리중
barrel 처리중
bass 처리중
beaver 처리중
binocular 처리중
bonsai 처리중
brain 처리중
brontosaurus 처리중
buddha 처리중
butterfly 처리중
camera 처리중
cannon 처리중
car_side 처리중
ceiling_fan 처리중
cellphone 처리중
chair 처리중
chandelier 처리중
cougar_body 처리중
cougar_face 처리중
crab 처리중
crayfish 처리중
crocodile 처리중
crocodile_head 처리중
cup 처리중
dalmatian 처리중
dollar_bill 처리중
dolphin 처리중
dragonfly 처리중
electric_guitar 처리중
elephant 처리중
emu 처리중
euphonium 처리중
ewer 처리중
Faces 처리중
Faces_easy 처리중
ferry 처리중
flamingo 처리중
flamingo_head 처리중
garfield 처리중
gerenuk 처리중
gramophone 처리중
grand_piano 처리중
hawksbill 처리중
headphone 처리중
hedgehog 처리중
helicopter 처리중
ibis 처리중
inline_skate 처리중
joshua_tree 처리중
kangaroo 처리중
ketch 처리중
lamp 처리중
laptop 처리중
Leopards 처리중
llama 처리중
lobster 처리중
lotus 처리중
mandolin 처리중
mayfly 처리중
menorah 처리중
metronome 처리중
minaret 처리중
Motorbikes 처리중
nautilus 처리중
octopus 처리중
okapi 처리중
pagoda 처리중
panda 처리중
pigeon 처리중
pizza 처리중
platypus 처리중
pyramid 처리중
revolver 처리중
rhino 처리중
rooster 

In [8]:
# ndarray를 생성한다.
X = np.array(X)
Y = np.array(Y)

In [9]:
# 학습 데이터와 테스트용 데이터로 나눈다.
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [10]:
# 저장한다.
xy = (X_train, X_test, y_train, y_test)
np.save('../images/caltech_new.npy', xy)
print(f'저장완료 : {len(Y)}')

저장완료 : 9145
